# EE499 Final Project
# Premier League Predictor
Hojeong Lee and Nate Chism

In [2]:
import numpy as np
import autograd.numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from autograd import grad 
from autograd import hessian
import math
import copy
import pandas as pd
import sys
import time
from IPython.display import clear_output
readDataPath = './readData/'
sys.path.append('./')

In [3]:
season_data = pd.read_csv('./readData/df_full_premierleague.csv')
season_csv_to_array = np.array(season_data)
# print(range(0, len(house_csv_to_array) -1))
print(season_csv_to_array.shape)
print(season_csv_to_array)

(4070, 114)
[[0 'https://www.premierleague.com/match/7186' '10/11' ... 0.78 -4.0 55.6]
 [1 'https://www.premierleague.com/match/7404' '10/11' ... 0.32 17.0 60.2]
 [2 'https://www.premierleague.com/match/7255' '10/11' ... 0.38 9.0 66.7]
 ...
 [4067 'https://www.premierleague.com/match/59178' '20/21' ... 0.69 4.0
  64.1]
 [4068 'https://www.premierleague.com/match/59182' '20/21' ... 0.42 6.0
  54.2]
 [4069 'https://www.premierleague.com/match/59052' '20/21' ... 0.87 5.0
  71.1]]


In [4]:
# 2, 4, 5, 6, 8 - 31, 38 - 49, 56  
print(season_csv_to_array[0][4])

Blackpool


In [11]:
i, j = season_csv_to_array.shape
print(i)
print(j)
match_target_scores = np.zeros((i,3))
# match_teams = np.array((i, 3))
for it_1 in range(0, i):
    # store game unique id
    match_target_scores[it_1][0] = season_csv_to_array[it_1][0]
    #store game unique id, home team, away team
    # match_teams[it_1][0] = season_csv_to_array[it_1][0]
    # match_teams[it_1][1] = season_csv_to_array[it_1][4]
    # match_teams[it_1][2] = season_csv_to_array[it_1][5]
    if season_csv_to_array[it_1][34] > 0:
        match_target_scores[it_1][1] = 1
    elif season_csv_to_array[it_1][34] < 0:
        match_target_scores[it_1][1] = -1
    else:
        match_target_scores[it_1][1] = 0

print(match_target_scores[678])

4070
114
[678.  -1.   0.]


In [6]:
season_data_cleaned = np.zeros((i, 102))


# 2, 4, 5, 6, 8 - 31, 38 - 49, 56  
for it in range(0, i):
    ct = 0
    for it2 in range(0, j):
        
        if it2 not in [1, 2, 3, 4, 5, 6, 7, 32, 33, 34, 35, 36, 37]:
            season_data_cleaned[it][ct] = season_csv_to_array[it][it2]
            # print(ct)
            ct = ct + 1

        elif it2 == 2:
            season_number = (int)(season_csv_to_array[it][it2][0] + season_csv_to_array[it][it2][1])
            season_data_cleaned[it][ct] = season_number
            ct = ct + 1
        

# print(season_data_cleaned)
print(season_data_cleaned[442])


[ 4.420e+02  1.100e+01  2.700e+01  7.000e+00  2.000e+01  1.000e+00
  2.620e+02  4.300e+01  0.000e+00  1.300e+01  4.000e+00  1.900e+01
  4.530e+02  3.000e+00  3.100e+01  8.000e+00  1.200e+01  6.000e+00
  3.710e+02  5.700e+01  0.000e+00  1.300e+01  6.000e+00  5.000e+00
  5.610e+02  2.000e+00  3.400e+01  5.100e+00  1.190e+01  1.700e+00
  3.711e+02  4.820e+01  2.000e-01  1.320e+01  4.900e+00  2.080e+01
  5.839e+02  1.000e+00  1.170e+00  2.080e+00 -1.100e+01  5.000e-01
  1.170e+00 -8.000e+00  2.220e+01  2.850e+01  4.200e+00  1.220e+01
  1.800e+00  3.952e+02  4.790e+01  2.000e-01  1.140e+01  3.400e+00
  2.080e+01  5.841e+02  1.400e+00  8.300e-01  2.250e+00 -1.700e+01
  4.200e-01  1.000e+00 -7.000e+00  2.220e+01  3.290e+01  5.000e+00
  1.150e+01  1.400e+00  3.691e+02  4.640e+01  2.000e-01  1.260e+01
  4.500e+00  2.020e+01  5.816e+02  1.400e+00  1.170e+00  2.040e+00
 -2.100e+01  5.000e-01  8.800e-01 -9.000e+00  2.780e+01  2.500e+01
  5.100e+00  1.310e+01  1.700e+00  4.105e+02  4.890e+01  1.000

In [7]:
print("it: ", it, " ct: ", ct, " it2: ", it2)

it:  4069  ct:  102  it2:  113


In [8]:
print(season_data_cleaned[4000][1])

20.0


In [15]:
season_data_cleaned[123][0]

123.0

In [22]:
seasons_10to18 = np.zeros((3419, 102))
season_19 = np.zeros((380, 102))
season_20 = np.zeros((271, 102))

Y_train = np.zeros((3419,))
Y_test1 = np.zeros((380,))
Y_test2 = np.zeros((271,))

ct1 = 0
ct2 = 0
ct3 = 0
for it in range(0, i):
    target_index = season_data_cleaned[it][0]
    if season_data_cleaned[it][1] < 19:
        seasons_10to18[ct1] = season_data_cleaned[it]
        Y_train[ct1] = match_target_scores[it][1]
        ct1 = ct1 + 1
    elif season_data_cleaned[it][1] == 19:
        season_19[ct2] = season_data_cleaned[it]
        Y_test1[ct2] = match_target_scores[it][1]
        ct2 = ct2 + 1
        # print(it)
    elif season_data_cleaned[it][1] == 20:
        season_20[ct3] = season_data_cleaned[it]
        Y_test2[ct3] = match_target_scores[it][1]
        ct3 = ct3 + 1
        # print(it)
    else:
        print('THIS SHOULD NEVER HAPPEN')

In [19]:
print(seasons_10to18.shape)
print(season_19.shape)
print(season_20.shape)

(3419, 102)
(380, 102)
(271, 102)


In [24]:
X_train = seasons_10to18
X_test1 = season_19
X_test2 = season_20



[-1.  1. -1.  1. -1. -1.  1.  0.  1.  1. -1. -1.  1.  1.  1. -1.  0.  1.
 -1.  1.  1.  1. -1. -1.  1.  0.  0. -1.  0.  0. -1.  1. -1.  1.  1. -1.
  1.  1.  0.  1. -1.  1. -1.  1. -1.  0. -1.  1.  1. -1. -1.  1.  1. -1.
  0.  0.  1. -1. -1.  1.  0. -1.  1.  1.  1. -1.  0.  1.  1. -1.  0.  1.
 -1.  0. -1.  1. -1.  1.  1. -1.  1.  0.  0.  1.  1.  0.  0.  1. -1. -1.
 -1. -1.  0. -1.  1.  0.  1.  0. -1.  1.  1.  1.  1.  0.  1.  0.  1. -1.
  1. -1.  0. -1.  1. -1.  0.  1.  1. -1.  1.  0.  0.  0.  1.  0. -1.  0.
 -1.  1.  1. -1.  1. -1. -1.  1.  1. -1.  0.  1.  1. -1. -1.  1. -1. -1.
  0.  0.  1. -1.  0. -1.  0.  0.  0.  1. -1. -1. -1.  1. -1. -1. -1. -1.
  1.  1. -1.  0.  1. -1. -1.  1.  0.  1.  0.  0.  0. -1. -1.  0.  1.  0.
  0. -1. -1.  1. -1.  0. -1. -1. -1.  1.  1. -1. -1.  1.  1. -1. -1.  1.
  1.  1. -1.  1. -1. -1. -1.  1.  0. -1. -1.  0. -1.  1.  0. -1.  0.  0.
 -1. -1. -1. -1. -1. -1. -1. -1.  1. -1.  0.  1.  1.  1.  1. -1.  1.  1.
  1.  1. -1. -1. -1. -1. -1.  1.  1.  0. -1.  1.  0